# Current Stock Momentum Calculation

This notebook calculates current momentum for all stocks in the Rice Business Stock Market Data Portal.

Momentum is defined as:
- (Price 21 trading days ago / Price 252 trading days ago) - 1

This captures approximately 11 months of returns (252 trading days ≈ 1 year, 21 trading days ≈ 1 month), skipping the most recent month to avoid short-term reversals.

## Setup

In [23]:
from rice_data_client import RiceDataClient
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment
ACCESS_TOKEN = os.getenv('USER_ACCESS_TOKEN')
BASE_URL = os.getenv('RICE_DATA_URL', 'https://portal.rice-business.org')

# Connect to Rice Data Portal
client = RiceDataClient(
    access_token=ACCESS_TOKEN,
    base_url=BASE_URL
)

print("Connected to Rice Data Portal")

✅ Connected to Rice Business Stock Market Data
📧 User: verified_user@rice.edu
🔑 Permissions: read_all, sql_queries
Connected to Rice Data Portal


## Find the Most Recent Trading Day

In [24]:
# Query to find the most recent trading day in SEP table
sql_recent = """
SELECT MAX(date::DATE) as max_date
FROM sep
"""

# Execute query using rice_data_client
result = client.query(sql_recent)

most_recent_date = pd.to_datetime(result['max_date'].iloc[0])
print(f"Most recent trading day: {most_recent_date.strftime('%Y-%m-%d')}")

📊 Query returned 1 rows
Most recent trading day: 1970-01-01


## Get Prices at Two Key Dates

We only need prices from two specific dates:
- Most recent date (for reference)
- 21 trading days ago (for momentum numerator)
- 252 trading days ago (for momentum denominator)

We'll identify these dates and query only those specific dates to avoid timeouts.

In [25]:
# Step 1: Get the most recent 253 unique trading dates from SEP table
# We need 253 dates to get: current date (index 0) and 252 days ago (index 252)
sql_dates = """
SELECT DISTINCT date
FROM sep
ORDER BY date DESC
LIMIT 253
"""

print("Identifying trading days...")
trading_dates_df = client.query(sql_dates)

# Convert to list of date strings, sorted descending
trading_dates = trading_dates_df['date'].tolist()

# Get the specific dates we need
date_current_str = trading_dates[0]      # Most recent date
date_lag21_str = trading_dates[21]       # 21 trading days ago  
date_lag252_str = trading_dates[252]     # 252 trading days ago

print(f"Most recent date: {date_current_str}")
print(f"Date 21 trading days ago: {date_lag21_str}")
print(f"Date 252 trading days ago: {date_lag252_str}")

# Step 2: Get adjusted closing prices for all stocks at these two dates only
sql = f"""
SELECT 
    ticker,
    date,
    closeadj
FROM sep
WHERE date IN (
    '{date_lag21_str}',
    '{date_lag252_str}'
)
ORDER BY ticker, date
"""

print(f"\nDownloading prices for 2 dates...")
df = client.query(sql)

print(f"Downloaded {len(df):,} observations for {df['ticker'].nunique():,} tickers")
df.head(10)

Identifying trading days...
📊 Query returned 253 rows
Most recent date: 2025-10-16
Date 21 trading days ago: 2025-09-17
Date 252 trading days ago: 2024-10-14

📊 Query returned 8692 rows
Downloaded 8,692 observations for 4,635 tickers


,ticker,date,closeadj
0,A,2024-10-14,143.100
1,A,2025-09-17,126.476
2,AA,2024-10-14,41.289
3,AA,2025-09-17,32.860
4,AACB,2025-09-17,10.130
5,AACI,2025-09-17,10.290
6,AACT,2024-10-14,10.830
7,AACT,2025-09-17,11.390
8,AAL,2024-10-14,11.870
9,AAL,2025-09-17,12.480


## Data Preparation

In [26]:
# Convert date to datetime and pivot to get prices at each date
df['date'] = pd.to_datetime(df['date'])

# Pivot so each ticker has one row with prices at the two dates
df_pivot = df.pivot(index='ticker', columns='date', values='closeadj')

# The columns will be the two dates - rename them for clarity
# The earlier date is price_lag252, the later date is price_lag21
cols = sorted(df_pivot.columns)
df_pivot.columns = ['price_lag252', 'price_lag21']

print(f"\nPivoted data: {len(df_pivot):,} tickers with prices at 2 dates")
print(f"\nFirst few rows:")
df_pivot.head()


Pivoted data: 4,635 tickers with prices at 2 dates

First few rows:


,price_lag252,price_lag21
ticker,,
A,143.100,126.476
AA,41.289,32.860
AACB,NaN,10.130
AACI,NaN,10.290
AACT,10.830,11.390


## Calculate Momentum

Momentum = (Price 21 trading days ago / Price 252 trading days ago) - 1

In [27]:
# Calculate momentum
df_pivot['momentum'] = (df_pivot['price_lag21'] / df_pivot['price_lag252']) - 1

# Remove tickers without complete data
df_current = df_pivot.dropna(subset=['momentum']).copy()
df_current = df_current.reset_index()

# Add the current date for reference (using date_current_str from earlier)
df_current['date'] = date_current_str

# Add closeadj as the most recent price (price_lag21 is the most recent we have)
df_current['closeadj'] = df_current['price_lag21']

# Keep only the columns we need
df_current = df_current[['ticker', 'date', 'closeadj', 'momentum']]

# Sort by momentum (descending)
df_current = df_current.sort_values('momentum', ascending=False)

print(f"\nMomentum calculated for {len(df_current):,} tickers")
print(f"Reference date: {date_current_str}")
print(f"Momentum period: {date_lag252_str} to {date_lag21_str}")
print(f"\nTop 10 momentum stocks:")
print(df_current[['ticker', 'closeadj', 'momentum']].head(10).to_string(index=False))
print(f"\nBottom 10 momentum stocks:")
print(df_current[['ticker', 'closeadj', 'momentum']].tail(10).to_string(index=False))


Momentum calculated for 4,057 tickers
Reference date: 2025-10-16
Momentum period: 2024-10-14 to 2025-09-17

Top 10 momentum stocks:
ticker  closeadj  momentum
  DFDV     17.91 25.572700
  RGTI     21.99 25.116390
  QUBT     17.71 23.427586
  QBTS     22.54 21.316832
  MVST      3.36 14.700935
   RYM     40.12 12.692833
  TDUP     10.45 12.312102
  STEX      6.21 12.212766
  MNPR     66.64 11.363636
  PRCH     18.09 11.222973

Bottom 10 momentum stocks:
ticker  closeadj  momentum
   CDT     7.191 -0.994487
  ACON     7.627 -0.995157
  ADTX     1.010 -0.996327
  XXII     1.740 -0.996453
  GCTK     7.550 -0.997087
  SUNE     1.420 -0.998862
  GNLN     3.570 -0.998911
  DGLY     1.950 -0.999145
  WHLR    10.400 -0.999387
  BINI    15.000 -1.000000


## Save to Excel

Save the current momentum data to Excel format for easy viewing and sharing.

In [29]:
# Save to Excel
# Create filename with date (replace hyphens with nothing for YYYYMMDD format)
date_str = df_current["date"].iloc[0].replace('-', '')
excel_filename = f'momentum_current_{date_str}.xlsx'

# Format momentum as percentage for better readability
# Prepare output data
output_excel = df_current[['ticker', 'date', 'momentum']].copy()
output_excel = output_excel.sort_values('momentum', ascending=False)
output_excel['momentum_pct'] = output_excel['momentum'] * 100

# Reorder columns
output_excel = output_excel[['ticker', 'date', 'momentum', 'momentum_pct']]

# Save to Excel with formatting
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    output_excel.to_excel(writer, sheet_name='Momentum', index=False)
    
    # Get the worksheet to apply formatting
    worksheet = writer.sheets['Momentum']
    
    # Set column widths
    worksheet.column_dimensions['A'].width = 12  # ticker
    worksheet.column_dimensions['B'].width = 12  # date
    worksheet.column_dimensions['C'].width = 14  # closeadj
    worksheet.column_dimensions['D'].width = 16  # momentum_pct
    worksheet.column_dimensions['E'].width = 18  # momentum_decile
    
    # Format header row
    for cell in worksheet[1]:
        cell.font = cell.font.copy(bold=True)

print(f"\nCurrent momentum data saved to {excel_filename}")
print(f"Total stocks: {len(output_excel):,}")
print(f"Date: {output_excel['date'].iloc[0]}")


Current momentum data saved to momentum_current_20251016.xlsx
Total stocks: 4,057
Date: 2025-10-16


C:\Users\kerry\AppData\Local\Temp\ipykernel_21352\3297156828.py:31: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.font = cell.font.copy(bold=True)


## Summary

This notebook:
1. Identified the most recent trading day in the Rice Data Portal (SEP table)
2. Downloaded daily prices from SEP going back ~400 calendar days
3. Calculated momentum for each stock as (Price_t-21 / Price_t-252) - 1
4. Extracted current momentum values as of the most recent trading day
5. Analyzed the distribution and created decile rankings
6. Saved the results to Parquet and Excel files

**Note**: The SEP table contains daily prices. We use trading day lags (21 and 252) rather than calendar days.

The momentum measure uses 21 trading days (≈1 month) and 252 trading days (≈1 year) to capture approximately 11 months of returns, skipping the most recent month to avoid short-term reversals.